In [1]:
# Dependencies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%matplotlib inline
%load_ext tensorboard

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from packaging import version
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow import keras
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
# Read exoplanet data
df = pd.read_csv("exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
df = df.dropna()
print(df.describe())
df.head()

       koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
count    6991.000000    6991.000000    6991.000000    6991.000000   
mean        0.157059       0.244743       0.202975       0.125018   
std         0.363882       0.429966       0.402243       0.330763   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         1.000000       1.000000       1.000000       1.000000   

        koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
count  6991.000000     6.991000e+03     6.991000e+03  6991.000000   
mean     56.191248     1.851122e-03    -1.851122e-03   164.488820   
std     117.570962     7.184503e-03     7.184503e-03    67.020475   
min       0.259820     1.100000e-08    -1.568000e-01   120.515914   
25%       2.620126     5.005000e-

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df.columns)
df_renamed=df.rename(columns={"koi_disposition":"Disposition",
                              "koi_fpflag_nt":"Not Transit Like",
                              "koi_fpflag_ss":"Stellar Eclipse",
                              "koi_fpflag_co":"Centroid Offset",
                              "koi_fpflag_ec":"Ephemeris Match Indicates Contamination",
                              "koi_period":"Orbital Period (Days)",
                              "koi_period_err1":"Orbital Period Positive Error",
                              "koi_period_err2":"Orbital Period Negative Error",
                              "koi_time0bk":"Transit Epoch (BJD - 2,454,833 Days)",
                              "koi_time0bk_err1":"Transit Epoch Positive Error",
                              "koi_time0bk_err2":"Transit Epoch Negative Error",
                              "koi_impact":"Impact",
                              "koi_impact_err1":"Impact Positive Error",
                              "koi_impact_err2":"Impact Negative Error",
                              "koi_duration":"Transit Duration (Hrs)",
                              "koi_duration_err1":"Transit Duration Postive Error",
                              "koi_duration_err2":"Transit Duration Negative Error",
                              "koi_depth":"Transit Depth (Parts/M)",
                              "koi_depth_err1":"Transit Depth Positive Error",
                              "koi_depth_err2":"Transit Depth Negative Error",
                              "koi_prad":"Planetary Radius (Earth Radii)",
                              "koi_prad_err1":"Planetary Radius Positive Error",
                              "koi_prad_err2":"Planetary Radius Negative Error",
                              "koi_teq":"Equilibrium Temperature (K)",
                              "koi_insol":"Insolation Flux (Earth Flux)",
                              "koi_insol_err1":"Insolation Flux Positive Error",
                              "koi_insol_err2":"Insolation Flux Negative Error",
                              "koi_model_snr":"Transit Signal-to-Noise",
                              "koi_tce_plnt_num":"TCE Planet Number",
                              "koi_steff":"Stellar Effective Temperature (K)",
                              "koi_steff_err1":"Stellar Effective Temperature Positive Error",
                              "koi_steff_err2":"Stellar Effective Temperature Negative Error",
                              "koi_slogg":"Stellar Surface Gravity",
                              "koi_slogg_err1":"Stellar Surface Gravity Positive Error",
                              "koi_slogg_err2":"Stellar Surface Gravity Negative Error",
                              "koi_srad":"Stellar Radius (Solar Radii)",
                              "koi_srad_err1":"Stellar Radius Positive Error",
                              "koi_srad_err2":"Stellar Radius Negative Error",
                              "ra":"KIC Right Ascension (Degrees)",
                              "dec":"KIC Declination (Degrees)",
                              "koi_kepmag":"Kepler-band (Magnitude)"
                  })
print(df_renamed.columns)
df_renamed.head()

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')
Index(['Disposition', 'Not Transit Like', 'Stellar Eclipse', 'Centroid Offset',
       'Ephemeris Match Indicates Contamination', 'Orbital Period (Days)',
       'Orbital Period Positive Error', 'Orbital Period Negative Error',
     

,Disposition,Not Transit Like,Stellar Eclipse,Centroid Offset,Ephemeris Match Indicates Contamination,Orbital Period (Days),Orbital Period Positive Error,Orbital Period Negative Error,"Transit Epoch (BJD - 2,454,833 Days)",Transit Epoch Positive Error,...,Stellar Effective Temperature Negative Error,Stellar Surface Gravity,Stellar Surface Gravity Positive Error,Stellar Surface Gravity Negative Error,Stellar Radius (Solar Radii),Stellar Radius Positive Error,Stellar Radius Negative Error,KIC Right Ascension (Degrees),KIC Declination (Degrees),Kepler-band (Magnitude)
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Preprocess Data
y = df_renamed["Disposition"]
X = df_renamed[[
    "Not Transit Like",
    "Stellar Eclipse",
    "Centroid Offset",
    "Ephemeris Match Indicates Contamination",
    "Impact",
    "Transit Duration (Hrs)",
    "Planetary Radius (Earth Radii)",
    "Stellar Surface Gravity",
    "Stellar Radius (Solar Radii)"
]]
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

(6991, 9) (6991,)


In [5]:
# Verify Tensorflow version
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2,"This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.1.0


In [6]:
# Clear logs
!rm -rf ./logs/

# Create Model
def create_model():
    model = Sequential()
    model.add(Dense(units=27, activation='relu', input_dim=9))
    model.add(Dense(units=27, activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    return model

model = create_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs\\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Fit Model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=15,
    shuffle=True,
    verbose=0,
    validation_data=(X_test_scaled, y_test_categorical),
    callbacks=[tensorboard_callback]
)

logs\fit\20200131-163857


In [7]:
# Quantify Trained Model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.4035 - accuracy: 0.7637
Normal Neural Network - Loss: 0.4035197197708985, Accuracy: 0.763729989528656


In [8]:
# Save Model
model.save("exoplanet_limited_hyperparameter_model_trained.h5")

In [10]:
# Start TensorBoard http://localhost:6006/
# Rerun cell if it times out
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 37560), started 0:00:45 ago. (Use '!kill 37560' to kill it.)

In [11]:
# print parameters
print(model.layers)
layer_num = 0
for layer in model.layers:
    print("Layer "+str(layer_num))
    print("Kernel")
    print(layer.get_weights()[0])
    print("Bias")
    print(layer.get_weights()[1])
    layer_num += 1

[<tensorflow.python.keras.layers.core.Dense object at 0x000001F65D792EB8>, <tensorflow.python.keras.layers.core.Dense object at 0x000001F65D7A9128>, <tensorflow.python.keras.layers.core.Dense object at 0x000001F65D7DE4A8>]
Layer 0
Kernel
[[-0.40376824  0.55908793 -0.04381289  0.48187754 -0.076156    0.13187186
   0.17160393  0.4089175   0.3946828  -0.67994994  0.22857474  0.25503144
   0.17465323 -0.14896834 -0.30161688  0.23935105  0.28805605 -0.12556997
   0.07501008  0.00994759 -0.00932811  0.24856265  0.04101009  0.05610132
   0.2073842  -0.19209284 -0.48959586]
 [-0.47973084 -0.25757694 -0.02873671 -0.20586155  0.05055105  0.25940442
  -0.2856574  -0.0759172   0.4228586  -0.31559896  0.3442584  -0.2486642
   0.09675357 -0.08184822 -0.38490593  0.47141653  0.42663166 -0.57604116
   0.44028848 -0.07151917 -0.3542329  -0.06053799  0.42375743  0.04580187
   0.12185733  0.13811836 -0.4704318 ]
 [-0.03120202 -0.11975237  0.5736648   0.05587358  0.3741502   0.34783518
  -0.28119433  0.55